In [1]:
from sqlalchemy import create_engine
import pandas as pd
from sqlalchemy import text
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem import WordNetLemmatizer
import re
import string
import pickle
from EDA import clean_ratings_dist, clean_ratings_dist_2, clean_gr_description, lower_clean, clean_description, word_tokenize_description, lemmatize_nouns, lemmatize_verbs, lemmatized_column

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/annabradleywebb/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
nltk.download('wordnet')
lemmatizer = nltk.stem.WordNetLemmatizer()

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/annabradleywebb/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
cnx = create_engine('postgresql://ubuntu@18.222.253.94:5432/books')

In [4]:
non_bestsellers_query = (''' 
select * from kagglegoodreads 
where description is not null 
and authors not in ('[NOT A BOOK]', '[McGraw-Hill%]')
and title not like '%%McGraw-Hill%%' 
and title not like '%%Instructional%%' 
and title not like '%%Fundamentals of%%' 
and title not like '%%Handbook%%'
and title not like '%%Education%%'
and ratings_count not like '0'
and publisher not like '%%University Press%%'
and authors not like '%%,%%,%%,%%'
and description not like '%%textbook%%'
and description like '%novel%' or description like '%story%'
and description not like '%%education%%'
and description not like '%%teacher%%'
''')

In [5]:
#Bestsellers df to match non-bestsellers
bestsellers = ('''
select
*
from 
nytgoodreads
''')

In [6]:
#Bestsellers df with full additional information
bestsellers_join = ('''
select sum("rank") as "sum_of_bestseller_rank", 
min("rank") as "best_nyt_rank", 
max(weeks_on_list) as "total_weeks_on_nyt_list", 
u_nyt_bestsellers.title,
u_nyt_bestsellers.author, 
min(nytgoodreads.description) as "goodreads_description", 
min(u_nyt_bestsellers.description) as "nyt_description",
nytgoodreads.avg_rating as "goodreads_avg_rating",
min(nytgoodreads.publication_date) as "earliest_publication", 
min(nytgoodreads.rating_dist) as "goodreads_rating_dist",
max(nytgoodreads.text_reviews_count) as "goodreads_reviews_count",
min(u_nyt_bestsellers.primary_isbn10) as "isbn_10", 
min(u_nyt_bestsellers.primary_isbn13) as "isbn_13",
case when sunday_review_link is not null then 1 else 0 end as "nyt_sunday_review",
case when book_review_link is not null then 1 else 0 end as "nyt_book_review",
case when first_chapter_link is not null then 1 else 0 end as "nyt_first_chapter_featured",
min("date") as "first_week_on_nyt_list",
max("date") as "last_week_on_nyt_list"
from u_nyt_bestsellers 
inner join nytgoodreads on text(nytgoodreads.isbn13) = u_nyt_bestsellers.primary_isbn13 
group by u_nyt_bestsellers.title, nytgoodreads.avg_rating, u_nyt_bestsellers.author,
"nyt_sunday_review", "nyt_book_review", "nyt_first_chapter_featured"
order by total_weeks_on_nyt_list desc
''')

In [7]:
bestsellers = pd.read_sql_query(bestsellers, cnx)

In [8]:
non_bestsellers = pd.read_sql_query(text(non_bestsellers_query), cnx)

In [9]:
non_bestsellers.shape

(5691, 17)

In [10]:
bestsellers.shape

(2869, 18)

In [11]:
non_bestsellers = non_bestsellers.drop(["index"],axis=1)

In [12]:
non_bestsellers["bestseller"] = 0

In [13]:
bestsellers["bestseller"] = 1

Separate rating distributions into columns

In [14]:
bestsellers = clean_ratings_dist(bestsellers, "rating_dist")

In [15]:
bestsellers = clean_ratings_dist_2(bestsellers, "rating_dist")

In [16]:
non_bestsellers = clean_ratings_dist(non_bestsellers, "rating_dist")

In [17]:
non_bestsellers = clean_ratings_dist_2(non_bestsellers, "rating_dist")

Take out HTML elements

In [18]:
bestsellers = clean_gr_description(bestsellers, "description")

/Users/annabradleywebb/PycharmProjects/Project5/EDA/EDA.py:34: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 34 of the file /Users/annabradleywebb/PycharmProjects/Project5/EDA/EDA.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  soup = BeautifulSoup(df[description_column][i])
/Users/annabradleywebb/PycharmProjects/Project5/EDA/EDA.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[description_column][i] = soup.get_text()


In [19]:
non_bestsellers = clean_gr_description(non_bestsellers, "description")

Remove non-alphabet characters and make text lowercase

In [20]:
bestsellers = clean_description(bestsellers, "description")

/Users/annabradleywebb/PycharmProjects/Project5/EDA/EDA.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[description_column][i] = lower_clean(df[description_column][i])


In [21]:
non_bestsellers = clean_description(non_bestsellers, "description")

In [22]:
bestsellers = clean_description(bestsellers, "title")

In [23]:
non_bestsellers = clean_description(non_bestsellers, "title")

Preparing for NLP on Goodreads description

In [24]:
bestsellers = word_tokenize_description(bestsellers, "description")

/Users/annabradleywebb/PycharmProjects/Project5/EDA/EDA.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['word_tokenized_description'][i] = nltk.word_tokenize(df[text][i])
/Users/annabradleywebb/anaconda3/envs/metis/lib/python3.7/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [25]:
non_bestsellers = word_tokenize_description(non_bestsellers, "description")

In [26]:
bestsellers = lemmatized_column(bestsellers)

/Users/annabradleywebb/PycharmProjects/Project5/EDA/EDA.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['lemmatized_description'][i] = lemmatize_nouns(df['word_tokenized_description'][i])
/Users/annabradleywebb/PycharmProjects/Project5/EDA/EDA.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['lemmatized_description'][i] = lemmatize_verbs(df['lemmatized_description'][i])
/Users/annabradleywebb/PycharmProjects/Project5/EDA/EDA.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/

In [27]:
non_bestsellers = lemmatized_column(non_bestsellers)

In [28]:
bestsellers

,index,gid,title,authors,description,avg_rating,ratings_count,text_reviews_count,num_pages,popular_shelves,...,link,bestseller,5-star,4-star,3-star,2-star,1-star,total,word_tokenized_description,lemmatized_description
0,0,7361506,alone detective d d warren,[Lisa Gardner],bonus this edition includes the full text of ...,3.96,106,32,324.0,"[to-read, currently-reading, mystery, lisa-gar...",...,https://www.goodreads.com/book/show/7361506-alone,1,15893,21149,11622,2035,696,51395,"[bonus, this, edition, includes, the, full, te...",bonus this edition include the full text of th...
1,0,10130768,tick tock,"[James Patterson, Michael Ledwidge]",nyc s detective michael bennett has a hug...,4.05,47,8,NaN,"[to-read, currently-reading, james-patterson, ...",...,https://www.goodreads.com/book/show/10130768-t...,1,8334,9946,4980,783,172,24215,"[nyc, s, detective, michael, bennett, has, a, ...",nyc s detective michael bennett ha a huge prob...
2,0,6620989,the girl with the dragon tattoo millennium,"[Stieg Larsson, Reg Keeland]",a spellbinding amalgam of murder mystery fami...,4.13,15960,3083,429.0,"[to-read, currently-reading, fiction, mystery,...",...,https://www.goodreads.com/book/show/6620989-th...,1,1136433,845761,355379,106672,75936,2520181,"[a, spellbinding, amalgam, of, murder, mystery...",a spellbind amalgam of murder mystery family s...
3,0,7654769,water for elephants,[Sara Gruen],over copies in print worldwide n...,4.09,1749,311,292.0,"[to-read, currently-reading, fiction, favorite...",...,https://www.goodreads.com/book/show/7654769-wa...,1,528848,499656,236299,57872,19760,1342435,"[over, copies, in, print, worldwide, new, york...",over copy in print worldwide new york time bes...
4,0,6652906,the girl who played with fire millennium,"[Stieg Larsson, Reg Keeland]",part blistering espionage thriller part rivet...,4.24,6407,773,471.0,"[to-read, currently-reading, fiction, mystery,...",...,https://www.goodreads.com/book/show/6652906-th...,1,350054,307130,106664,18618,6123,788589,"[part, blistering, espionage, thriller, part, ...",part blister espionage thriller part rivet pol...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2864,0,51180570,hello summer,[Mary Kay Andrews],new york times bestselling author and queen of...,4.18,1151,356,558.0,"[to-read, currently-reading, netgalley, fictio...",...,https://www.goodreads.com/book/show/51180570-h...,1,817,936,314,38,16,2121,"[new, york, times, bestselling, author, and, q...",new york time bestselling author and queen of ...
2865,0,52383564,network effect the murderbot diaries,[Martha Wells],you know that feeling when you re at work and...,4.61,65,21,352.0,"[to-read, currently-reading, science-fiction, ...",...,https://www.goodreads.com/book/show/52383564-n...,1,3951,1580,293,28,7,5859,"[you, know, that, feeling, when, you, re, at, ...",you know that feel when you re at work and you...
2866,0,52879371,the th victim,"[James Patterson, Maxine Paetro]",three victims three bullets three cities th...,4.21,942,98,NaN,"[to-read, currently-reading, james-patterson, ...",...,https://www.goodreads.com/book/show/52879371-t...,1,1678,1353,619,98,28,3776,"[three, victims, three, bullets, three, cities...",three victim three bullet three city the shoot...
2867,0,53419239,the last trial,[Scott Turow],in this explosive legal thriller from new york...,4.11,3,1,485.0,"[to-read, currently-reading, fiction, mystery,...",...,https://www.goodreads.com/book/show/53419239-t...,1,431,362,161,53,18,1025,"[in, this, explosive, legal, thriller, from, n...",in this explosive legal thriller from new york...


In [29]:
non_bestsellers.to_pickle('non_bestsellers.pkl')

In [30]:
bestsellers.to_pickle('bestsellers.pkl')

In [31]:
all_books = pd.concat([bestsellers, non_bestsellers])

In [32]:
all_books

,index,gid,title,authors,description,avg_rating,ratings_count,text_reviews_count,num_pages,popular_shelves,...,link,bestseller,5-star,4-star,3-star,2-star,1-star,total,word_tokenized_description,lemmatized_description
0,0.0,7361506,alone detective d d warren,[Lisa Gardner],bonus this edition includes the full text of ...,3.96,106,32,324,"[to-read, currently-reading, mystery, lisa-gar...",...,https://www.goodreads.com/book/show/7361506-alone,1,15893,21149,11622,2035,696,51395,"[bonus, this, edition, includes, the, full, te...",bonus this edition include the full text of th...
1,0.0,10130768,tick tock,"[James Patterson, Michael Ledwidge]",nyc s detective michael bennett has a hug...,4.05,47,8,NaN,"[to-read, currently-reading, james-patterson, ...",...,https://www.goodreads.com/book/show/10130768-t...,1,8334,9946,4980,783,172,24215,"[nyc, s, detective, michael, bennett, has, a, ...",nyc s detective michael bennett ha a huge prob...
2,0.0,6620989,the girl with the dragon tattoo millennium,"[Stieg Larsson, Reg Keeland]",a spellbinding amalgam of murder mystery fami...,4.13,15960,3083,429,"[to-read, currently-reading, fiction, mystery,...",...,https://www.goodreads.com/book/show/6620989-th...,1,1136433,845761,355379,106672,75936,2520181,"[a, spellbinding, amalgam, of, murder, mystery...",a spellbind amalgam of murder mystery family s...
3,0.0,7654769,water for elephants,[Sara Gruen],over copies in print worldwide n...,4.09,1749,311,292,"[to-read, currently-reading, fiction, favorite...",...,https://www.goodreads.com/book/show/7654769-wa...,1,528848,499656,236299,57872,19760,1342435,"[over, copies, in, print, worldwide, new, york...",over copy in print worldwide new york time bes...
4,0.0,6652906,the girl who played with fire millennium,"[Stieg Larsson, Reg Keeland]",part blistering espionage thriller part rivet...,4.24,6407,773,471,"[to-read, currently-reading, fiction, mystery,...",...,https://www.goodreads.com/book/show/6652906-th...,1,350054,307130,106664,18618,6123,788589,"[part, blistering, espionage, thriller, part, ...",part blister espionage thriller part rivet pol...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5686,NaN,18123696,the secret of abdu el yezdi burton swinburn...,"[Mark Hodder, Gerard Doyle]",burton swinburne return in a new series the...,3.93,14,2,None,NaN,...,https://www.goodreads.com/book/show/18123696-t...,0,268,405,247,36,7,963,"[burton, swinburne, return, in, a, new, series...",burton swinburne return in a new series the be...
5687,NaN,18100299,dark city,[F. Paul Wilson],repairman jack is one of my favorite characte...,4.19,8,0,None,NaN,...,https://www.goodreads.com/book/show/18100299-d...,0,368,414,148,15,4,949,"[repairman, jack, is, one, of, my, favorite, c...",repairman jack be one of my favorite character...
5688,NaN,13529366,breed,"[Chase Novak, Peter Ganim]",critically celebrated novelist scott spencer d...,3.15,7,2,12,NaN,...,https://www.goodreads.com/book/show/13529366-b...,0,379,1099,1501,708,270,3957,"[critically, celebrated, novelist, scott, spen...",critically celebrate novelist scott spencer de...
5689,NaN,31806710,defy the dawn midnight breed,"[Lara Adrian, Hillary Huber]",hours minutesan immortal warrior must dec...,4.14,22,4,None,NaN,...,https://www.goodreads.com/book/show/31806710-d...,0,1162,1092,518,91,22,2885,"[hours, minutesan, immortal, warrior, must, de...",hour minutesan immortal warrior must decide be...


In [33]:
all_books.to_pickle('all_books.pkl')